
# Alpha Zero for Connect Four

### Game Representation
Players: -1, 1

Empty spaces: 0

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random
import collections

In [2]:
class connect_four_game():
    def __init__(self,n,m):
        self.board_state = np.zeros((n,m)) # empty board of n rows and m columns
        self.action_space = []
        self.n = n
        self.m = m
        # NN input
        self.channel_0 = np.zeros((1,n,m))
        self.channel_1 = np.zeros((1,n,m))
        self.channel_2 = np.zeros((1,n,m))
    def clear_board(self):
        self.board_state = np.zeros((self.n,self.m))
        self.channel_0 = np.zeros((1,self.n,self.m))
        self.channel_1 = np.zeros((1,self.n,self.m))
        self.channel_2 = np.zeros((1,self.n,self.m))
    def find_actions(self):
        self.action_space = []
        for col in range(self.m):
            for row in reversed(range(self.n)):
                if self.board_state[row,col] == 0:
                    self.action_space.append([row,col])
                    break
    def make_move(self,player,drop_col):
        self.find_actions()
        for action in self.action_space:
            if action[1] == drop_col:
                self.board_state[action[0],action[1]] = player
                self.update_state(action[0],action[1],player)
                return 1
        else:
            return -1
        
    def check_winner(self,player):
        # four ways to win: four horizontal, four vertical, four diagonal left, four diagonal right
        non_zero_count = 0
        for col in range(self.m):
            for row in range(self.n):
                if self.board_state[row,col] != 0:
                    non_zero_count += 1
                if self.board_state[row,col] == player:
                    if row <= self.n-4: # enough room to connect four vertical
                        # check for four in a row verticle
                        if self.board_state[row+1,col] == player\
                        and self.board_state[row+2,col] == player\
                        and self.board_state[row+3,col] == player:
                            return 1
                        if col >= self.m-4:
                            # check for four in a row left diagonal down
                            if self.board_state[row+1,col-1] == player\
                            and self.board_state[row+2,col-2] == player\
                            and self.board_state[row+3,col-3] == player:
                                return 1   
                    if row >= self.n-4:
                        if col <= self.m-4: # enough room to connect four horizontal
                            # check for four in a row right diagonal up
                            if self.board_state[row-1,col+1] == player\
                            and self.board_state[row-2,col+2] == player\
                            and self.board_state[row-3,col+3] == player:
                                return 1  
                        if col >= self.m-4:
                            # check for four in a row left diagonal up
                            if self.board_state[row-1,col-1] == player\
                            and self.board_state[row-2,col-2] == player\
                            and self.board_state[row-3,col-3] == player:
                                return 1 
                    if col <= self.m-4: # enough room to connect four horizontal
                        # check for four in a row horizontal
                        if self.board_state[row,col+1] == player\
                        and self.board_state[row,col+2] == player\
                        and self.board_state[row,col+3] == player:
                            return 1
                        if row <= self.n-4: # enough room to connect four vertical
                            # check for four in a row right diagonal down
                            if self.board_state[row+1,col+1] == player\
                            and self.board_state[row+2,col+2] == player\
                            and self.board_state[row+3,col+3] == player:
                                return 1
        if non_zero_count == self.n*self.m:
            return 0
        return -1
    
    def update_state(self,row,col,player):
        if self.board_state[row,col] == 1:
            self.channel_0[0,row,col] = 1
        elif self.board_state[row,col] == -1:  
            self.channel_1[0,row,col] = 1
        if player == 1:
            self.channel_2 = np.ones((1,self.n,self.m))
        elif player == -1:
            self.channel_2 = np.ones((1,self.n,self.m))*-1
        
                                                    
    def play_game(self,first_player):
        self.clear_board()
        player = first_player
        game_over = -1
        while game_over == -1:
            drop_col = input("Enter drop column for Player " + str(player) + ": ")
            move_success = self.make_move(player,int(drop_col))
            while move_success == -1:
                drop_col = input("Enter drop column for Player " + str(player) + ": ")
                move_success = self.make_move(player,int(drop_col))
            self.print_board()
            game_over = self.check_winner(player)
            if game_over == 1:
                print("Game over player " + str(player) + " won!")
            elif game_over == 0:
                print("Game over no one won")
            if player == -1:
                player = 1
            else:
                player = -1

            
            
    def print_board(self):
        print(self.board_state)

                

### Convolutional NN

Input: state of the board: image of player 1 locations, image of player 2 locations, image of current player value (1 or -1)
* input nodes: for an 8 step history, 2 players, and color feature, there will be a nxmx17 stack input


Output: continuous value of the board state for the current player, policy probability vector
* output nodes: m possible starting positions to move and add another layer to normalize results into probability distribution


conv block -> 35 ResNets -> dense layer (fully connected) -> policy and value head (adjust as needed)


7x7 conv, 64, /2 -> pool,/2 -> 3x3 conv, 64 (x6) -> 3x3 conv, 128,/2 -> 3x3 conv, 128 (28)


see https://www.biostat.wisc.edu/~craven/cs760/lectures/AlphaZero.pdf for more info


CNN example: https://adventuresinmachinelearning.com/introduction-resnet-tensorflow-2/

In [3]:
# build the network
# Multi-head model
class CNN():
    def __init__(self,game):
        self.n = game.n # rows
        self.m = game.m # columns
        game.find_actions()
        self.actions = game.action_space
        self.state = np.concatenate((game.channel_0,game.channel_1,game.channel_2),axis=0)
        self.pi = np.zeros((len(game.action_space),1))
        self.value = 0
        self.model = 0
        self.prob = 0
    
    # Functional Mult-head model
    def ReturnModelFunctionalMultiHead(self):
        Inputshape = (3, self.n, self.m)
        output_dim_value = 1
        output_dim_pi = len(self.actions)

        model_inputs = tf.keras.Input(shape=Inputshape)

        # Pass input layer by layer, construct model from input to output
        x = tf.keras.layers.Conv2D(
                filters=64,
                kernel_size=(7,7),
                strides=(1, 1),
                padding='same',
                activation='relu')(model_inputs) # can also use relu for activiation
        x = tf.keras.layers.Conv2D(
                filters=64,
                kernel_size=(7,7),
                strides=(1, 1),
                padding='same',
                activation='relu')(x) # can also use relu for activiation
        
        x = tf.keras.layers.AveragePooling2D(
                pool_size=(2, 2),
                padding='valid')(x)
        
        x = tf.keras.layers.Conv2D(
                filters=64,
                kernel_size=(1,1),
                strides=(1, 1),
                padding='valid',
                activation='relu')(x)
        x = tf.keras.layers.Conv2D(
                filters=64,
                kernel_size=(1,1),
                strides=(1, 1),
                padding='valid',
                activation='relu')(x)

        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(
                units=120,
                activation='relu')(x)
        x = tf.keras.layers.Dense(
                units=84,
                activation='relu')(x)

        # Multi
        self.value = tf.keras.layers.Dense(
                units=output_dim_value,
                activation='tanh',
                name="head_value")(x)
        self.pi = tf.keras.layers.Dense(
                units=output_dim_pi,
                activation='softmax',
                name="head_pi")(x)
        self.model = tf.keras.Model(model_inputs, [self.value, self.pi]) # list of outputs
        self.model.compile(
        optimizer='sgd',
        loss=[
            tf.keras.losses.MeanSquaredError(), # two different loss functions as an example (do this for project)
            tf.keras.losses.CategoricalCrossentropy() # Could also use two of the same loss function
            ],
            loss_weights=[1.0, 1.0], # determines which head is valued more
        )
        self.model.summary()
        return self.model
    
    def fit(self,state,value,pi):
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='./saved_model/multi/',
        save_weights_only=True,
        monitor='val_loss',
        mode='max',
        save_best_only=True)
        curr_state = np.concatenate(state, axis=0)
        value_state = np.array([np.concatenate(value, axis=0) for i in range(curr_state.shape[0])])
        pi_state = np.concatenate(pi, axis=0)
        print(curr_state.shape)
        print(value_state.shape)
        print(pi_state.shape)
        self.model.fit(
            x = curr_state,
            #x=state, # states 
            y=[value_state, pi_state], # make it a list of values (value list w same length as x)
            validation_split = 0.2,
            batch_size=64,
            epochs=20,
            callbacks=[model_checkpoint_callback]
            )
        self.model.load_weights('./saved_model/multi/')
    
    def predict(self,pred_state):
        #Y_head_1_train = np.copy(Y_train)
        #Y_head_2_train = np.copy(Y_train)

        #Y_head_1_test = np.copy(Y_test)
        #Y_head_2_test = np.copy(Y_test)
        pred_state = pred_state.reshape([-1,3,self.n,self.m])
        self.value = self.model.predict(pred_state)[0]
        self.pi = [float(i[0]) for i in zip(*self.model.predict(pred_state)[1])]
        return self.value, self.pi
        
    def accuracy(self,test_state,true_value,true_pi):
        pred_state = test_state.reshape([-1,3,self.n,self.m])
        predv_head_1 = self.model.predict(pred_state)[0]
        truev = true_value
        total_acc = 0
        acc = 0
        for i in range(len(predv_head_1)):
            if predv_head_1[i] == true_value[i]:
                acc += 1
        total_acc += acc/len(predv_head_1)
        print("acc is : {}".format(acc/len(predv_head_1)))
        
        predv_head_2 = [float(i[0]) for i in zip(*self.model.predict(pred_state)[1])] # cross-entropy is better for policy head
        truev = true_pi
        acc = 0
        for i in range(len(predv_head_2)):
            if predv_head_2[i] == true_pi[i]:
                acc += 1
        total_acc += acc/len(predv_head_2)
        print("acc is : {}".format(acc/len(predv_head_2)))
        return total_acc
        


In [4]:
new_game = connect_four_game(6,7)
network = CNN(new_game)
network.ReturnModelFunctionalMultiHead()
network.state.shape

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 6, 7)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 3, 6, 64)     22016       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3, 6, 64)     200768      conv2d[0][0]                     
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 1, 3, 64)     0           conv2d_1[0][0]                   
______________________________________________________________________________________________

(3, 6, 7)

In [173]:
new_game.play_game(-1)

Enter drop column for Player -1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]]
Enter drop column for Player 1: 1
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]]
Enter drop column for Player -1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]]
Enter drop column for Player 1: 1
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]]
Enter drop column for Player -1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.

In [174]:
np.concatenate((new_game.channel_0,new_game.channel_1,new_game.channel_2),axis=0)

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[-1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1.]]])

In [179]:
network.predict(np.concatenate((new_game.channel_0,new_game.channel_1,new_game.channel_2),axis=0))

(array([[0.]], dtype=float32),
 [0.1428571492433548,
  0.1428571492433548,
  0.1428571492433548,
  0.1428571492433548,
  0.1428571492433548,
  0.1428571492433548,
  0.1428571492433548])

# MCTS

### Self Play

Create a training set
For each move, store:
- the game state
- the search probabilities (MCTS)
- the winner (1 for win, -1 for loss)

In [5]:
new_game = connect_four_game(6,7)
network = CNN(new_game)
network.ReturnModelFunctionalMultiHead()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 3, 6, 7)]    0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 3, 6, 64)     22016       input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 3, 6, 64)     200768      conv2d_4[0][0]                   
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 1, 3, 64)     0           conv2d_5[0][0]                   
____________________________________________________________________________________________

In [18]:
class MCTS():
    def __init__(self):
        self.explored = []
        self.Q = collections.defaultdict(dict) # Expected reward for taking the action from the current state
        self.p = {} # Initial estimate of taking an action from the current state according to the policy returned by the current NN
        self.N = collections.defaultdict(dict) # The number of times the action was taken from the current state accross simulations
        self.v = 0 # value from the neural network v E [-1,1]
        self.u = 0 # upper confidence bound
        self.c = 1 # hyperparameter for degree of exploration
        self.full_state = []
                
    def search(self,state,game,cnn,player):
        player_choice = game.check_winner(player)
        player_adverse = game.check_winner(int(player)*-1)
        if player_choice == 1:
            return player_choice # choice player won: 1
        elif player_adverse == 1:
            return -player_adverse # adverse player won: -1
        elif player_choice == 0:
            return 0 # tie: 0
            
        curr_state = game.board_state # get board configuration
        if str(curr_state) not in self.explored:
                self.explored.append(curr_state) # initialize the current state in the search tree
                self.full_state.append(np.concatenate((game.channel_0,game.channel_1,game.channel_2)))
                self.v,self.p[str(curr_state)] = cnn.predict(state) # set policy and value for state
                #game.find_actions()
                #valid_moves = [new_game.action_space[i][1] for i in range(len(new_game.action_space))]
                #self.p[str(curr_state)] = [self.p[str(curr_state)][i] for i in valid_moves]
                #if sum(self.p[str(curr_state)])>0:
                #    self.p[str(curr_state)] = self.p[str(curr_state)]/sum(self.p[str(curr_state)])
                #else:
                #    self.p[str(curr_state)] = self.p[str(curr_state)]+np.ones(len(self.p[str(curr_state)]))
                #    self.p[str(curr_state)]/=np.sum(self.p[str(curr_state)])
                return -self.v
        # determine the actions that maximizes the upper confidence bound, u
        u_max, best_action = -np.inf, -1
        game.find_actions()
        for action in game.action_space:
            action = action[1] # action is column choice
            self.u = self.Q[curr_state][action] + self.c*self.p[curr_state][action]*sqrt(sum\
            (self.N[curr_state]))/\
            (1+self.N[curr_state][action]) # upper confidence bound on the Q-values
            if self.u > u_max:
                u_max = self.u
                best_action = action
        action = best_action

        game.make_move(player,action)
        next_state = np.concatenate((new_game.channel_0,new_game.channel_1,new_game.channel_2))
        self.v = search(next_state,game,cnn)
        self.Q[curr_state][action] = (self.N[curr_state][action]*self.Q[curr_state][action]+self.v)/\
        (self.N[curr_state][action]+1)
        self.N[curr_state][action] += 1
        return -v
    
    def policyIteration(self,game,cnn,player):
        boards = []
        values = []
        pis = []
        for i in range(50): # iterations
            boards = []
            values = []
            pis = []
            for j in range(1): # episodes
                board_set,value_set,pi_set = self.selfPlay(game,cnn,player) # self play
                boards.append(board_set)
                values.append(value_set)
                pis.append(pi_set)
                game = connect_four_game(6,7)
            trained_cnn = cnn.fit(boards,values,pis)
        return trained_cnn
    
    def pitNNs(self,cnn_prev,cnn):
        wins_prev = 0
        wins_cnn = 0
        for i in range(10):
            game = connect_four_game(6,7)
            player = random.choice([-1,1])
            board_prev,v_prev,p_prev = self.selfPlay(game,cnn_prev,player)
            
    def win_rate(self,cnn):
        wins = 0
        N = 10
        for i in range(N):
            game = connect_four_game(6,7)
            player = random.choice([-1,1])
            board,v,p = self.selfPlay(game,cnn,player)
            if v[0] == 1:
                wins += 1
        return wins/N
    
    def selfPlay(self,game,cnn,player):
        boards = []
        values = []
        pis = []
        state = np.concatenate((game.channel_0,game.channel_1,game.channel_2),axis=0)
        curr_state = game.board_state # get board configuration
        game_over = -1
        first_player = player
        
        while game_over == -1:
            for i in range(10):
                self.search(state,game,cnn,player)
            policy = self.p[str(curr_state)]
            #games.append([state,policy,None])
            boards.append(state)
            pis.append(policy)
            print(policy)
            game.find_actions()
            print(game.action_space)
            action_cols = [game.action_space[i][1]-1 for i in range(len(game.action_space))]
            policy_act = [policy[i] for i in action_cols]
            action = random.choices(game.action_space,weights=policy_act)
            action = action[0]
            print(action)
            game.make_move(player,action[1])
            print(game.board_state)
            state = np.concatenate((game.channel_0,game.channel_1,game.channel_2),axis=0)
            curr_state = game.board_state # get board configuration
            player_choice = game.check_winner(int(first_player))
            player_adverse = game.check_winner(int(first_player)*-1)
            if player_choice == 1:
                values.append(player_choice)
                return boards,values,pis # choice player won: 1
            elif player_adverse == 1:
                values.append(-player_adverse)
                return boards,values,pis # adverse player won: -1
            elif player_choice == 0:
                values.append(0)
                return boards,values,pis # tie: 0
            if player == -1:
                player = 1
            else:
                player = -1
        
        
    def play_human(self,game,cnn,human_player):
        boards = []
        values = []
        pis = []
        state = np.concatenate((game.channel_0,game.channel_1,game.channel_2),axis=0)
        curr_state = game.board_state # get board configuration
        game_over = -1
        first_player = random.randint(-1, 1)
        computer = -1*int(human_player)
        player = first_player
    
        while game_over == -1:
            
            if player == computer: # computer plays
                for i in range(10):
                    self.search(state,game,cnn,player)
                policy = self.p[str(curr_state)]
                #games.append([state,policy,None])
                boards.append(state)
                pis.append(policy)
                print(policy)
                game.find_actions()
                print(game.action_space)
                action_cols = [game.action_space[i][1]-1 for i in range(len(game.action_space))]
                policy_act = [policy[i] for i in action_cols]
                action = random.choices(game.action_space,weights=policy_act)
                action = action[0]
                print(action)
                game.make_move(player,action[1])
                
            else: # human plays
                drop_col = input("Enter drop column for Player " + str(player) + ": ")
                move_success = game.make_move(player,int(drop_col))
                while move_success == -1:
                    drop_col = input("Enter drop column for Player " + str(player) + ": ")
                    move_success = game.make_move(player,int(drop_col))
                    
            print(game.board_state)
            state = np.concatenate((game.channel_0,game.channel_1,game.channel_2),axis=0)
            curr_state = game.board_state # get board configuration
            player_choice = game.check_winner(int(first_player))
            player_adverse = game.check_winner(int(first_player)*-1)
            if player_choice == 1:
                values.append(player_choice)
                return boards,values,pis # choice player won: 1
            elif player_adverse == 1:
                values.append(-player_adverse)
                return boards,values,pis # adverse player won: -1
            elif player_choice == 0:
                values.append(0)
                return boards,values,pis # tie: 0
            if player == -1:
                player = 1
            else:
                player = -1


        


In [19]:

mcts = MCTS()


In [383]:
ex_game = connect_four_game(6,7)
ex_game.play_game(-1)

Enter drop column for Player -1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]]
Enter drop column for Player 1: 1
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]]
Enter drop column for Player -1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]]
Enter drop column for Player 1: 1
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]]
Enter drop column for Player -1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.

In [306]:
mcts.search(np.concatenate((ex_game.channel_0,ex_game.channel_1,ex_game.channel_2),axis=0),new_game,network,-1)

array([[-0.00682186]], dtype=float32)

In [438]:
new_game = connect_four_game(6,7)
state,value,pi = mcts.selfPlay(new_game,network,-1)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


[0.14321313798427582, 0.14412978291511536, 0.14227421581745148, 0.14142997562885284, 0.1453661173582077, 0.14163018763065338, 0.14195653796195984]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[5, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]]
[0.14315803349018097, 0.14439022541046143, 0.14272059500217438, 0.1396346390247345, 0.1460149735212326, 0.14244329929351807, 0.1416381597518921]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [4, 5], [5, 6]]
[4, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]]
[0.14085184037685394, 0.14446806907653809, 0.14155004918575287, 0.14122140407562256, 0.14863114058971405, 0.14205607771873474, 0.14122144877910614]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [3, 5], [5, 6]]
[

[0.1422143131494522, 0.14443062245845795, 0.14266812801361084, 0.13956250250339508, 0.14779657125473022, 0.14224781095981598, 0.1410801261663437]
[[1, 1], [4, 2], [2, 3], [4, 4], [1, 5], [3, 6]]
[4, 4]
[[-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0. -1.  0.]
 [ 1. -1.  0.  1.  0. -1.  0.]
 [-1.  1.  0. -1.  1.  1. -1.]
 [ 1. -1.  1. -1. -1. -1.  1.]]
[0.1412539929151535, 0.1438017040491104, 0.14107008278369904, 0.14063242077827454, 0.1491575837135315, 0.1426285207271576, 0.14145563542842865]
[[1, 1], [4, 2], [2, 3], [3, 4], [1, 5], [3, 6]]
[2, 3]
[[-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0. -1.  0. -1.  0.]
 [ 1. -1.  0.  1.  0. -1.  0.]
 [-1.  1.  0. -1.  1.  1. -1.]
 [ 1. -1.  1. -1. -1. -1.  1.]]
[0.14242081344127655, 0.1443697065114975, 0.14269959926605225, 0.1399444341659546, 0.14753125607967377, 0.14204424619674683, 0.1409899741411209]
[[1, 1], [4, 2], [1, 3], [3, 4], [1, 5], [3, 6]]
[4, 2]
[[-1.  0.  0.  0.  0.  

In [440]:
print(state[-1])
print(value[-1])
print(pi[-1])

[[[ 0.  1.  0.  1.  1.  0.  0.]
  [ 1.  0.  0.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  1.  0.  0.]
  [ 1.  0.  0.  1.  0.  0.  0.]
  [ 0.  1.  1.  0.  1.  1.  0.]
  [ 1.  0.  1.  0.  0.  0.  1.]]

 [[ 1.  0.  0.  0.  0.  0.  0.]
  [ 0.  1.  1.  0.  1.  0.  0.]
  [ 0.  0.  1.  1.  0.  1.  0.]
  [ 0.  1.  1.  0.  1.  1.  0.]
  [ 1.  0.  0.  1.  0.  0.  1.]
  [ 0.  1.  0.  1.  1.  1.  0.]]

 [[-1. -1. -1. -1. -1. -1. -1.]
  [-1. -1. -1. -1. -1. -1. -1.]
  [-1. -1. -1. -1. -1. -1. -1.]
  [-1. -1. -1. -1. -1. -1. -1.]
  [-1. -1. -1. -1. -1. -1. -1.]
  [-1. -1. -1. -1. -1. -1. -1.]]]
-1
[0.14177925884723663, 0.14499622583389282, 0.14297160506248474, 0.14007209241390228, 0.1471308320760727, 0.1423957496881485, 0.14065416157245636]


In [439]:
np.shape(curr_state)

(42, 6, 7)

In [64]:
new_game.find_actions()
moves = [new_game.action_space[i][1] for i in range(len(new_game.action_space))]
moves

[1, 2, 3, 4, 5, 6]

In [11]:
new_game = connect_four_game(6,7)
mcts.policyIteration(new_game,network,1)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


[0.1412096917629242, 0.1442108452320099, 0.14320580661296844, 0.1395954191684723, 0.14775864779949188, 0.1427258998155594, 0.14129367470741272]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[5, 6]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
[0.1399887204170227, 0.1443924754858017, 0.14303210377693176, 0.13810737431049347, 0.15085481107234955, 0.14351259171962738, 0.14011196792125702]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [4, 6]]
[5, 4]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  1.]]
[0.1408788561820984, 0.1447344720363617, 0.14283578097820282, 0.13782252371311188, 0.15120160579681396, 0.14264926314353943, 0.13987751305103302]
[[5, 0], [5, 1], [5, 2], [5, 3], [4, 4], [5, 5], [4, 6]]
[5, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.

[0.13974052667617798, 0.14402927458286285, 0.14325779676437378, 0.13762502372264862, 0.15163780748844147, 0.143896222114563, 0.1398133784532547]
[[5, 0], [2, 1], [3, 2], [2, 3], [1, 4], [2, 6]]
[2, 3]
[[ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0. -1.  1.  1.  0.]
 [ 0.  1.  0. -1.  1. -1. -1.]
 [ 0. -1. -1. -1.  1. -1.  1.]
 [ 0.  1. -1.  1. -1.  1.  1.]]
[0.14003725349903107, 0.14463192224502563, 0.14287889003753662, 0.13814276456832886, 0.1516588181257248, 0.14247924089431763, 0.14017106592655182]
[[5, 0], [2, 1], [3, 2], [1, 3], [1, 4], [2, 6]]
[2, 1]
[[ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  1.  0. -1.  1.  1.  0.]
 [ 0.  1.  0. -1.  1. -1. -1.]
 [ 0. -1. -1. -1.  1. -1.  1.]
 [ 0.  1. -1.  1. -1.  1.  1.]]
[0.13968271017074585, 0.14400523900985718, 0.1432337462902069, 0.13763317465782166, 0.1516840159893036, 0.14387823641300201, 0.13988278806209564]
[[5, 0], [1, 1], [3, 2], [1, 3], [1, 4], [2, 6]]
[1, 4]
[[ 0.  0.  0.  0.  0

Epoch 10/20
30/30 [==============================] - 0s 806us/sample - loss: 2.5853 - head_value_loss: 0.6395 - head_pi_loss: 1.9458 - val_loss: 2.5402 - val_head_value_loss: 0.5944 - val_head_pi_loss: 1.9458
Epoch 11/20
30/30 [==============================] - 0s 799us/sample - loss: 2.5443 - head_value_loss: 0.5985 - head_pi_loss: 1.9458 - val_loss: 2.5030 - val_head_value_loss: 0.5572 - val_head_pi_loss: 1.9458
Epoch 12/20
30/30 [==============================] - 0s 849us/sample - loss: 2.5075 - head_value_loss: 0.5616 - head_pi_loss: 1.9458 - val_loss: 2.4687 - val_head_value_loss: 0.5228 - val_head_pi_loss: 1.9459
Epoch 13/20
30/30 [==============================] - 0s 831us/sample - loss: 2.4731 - head_value_loss: 0.5272 - head_pi_loss: 1.9459 - val_loss: 2.4356 - val_head_value_loss: 0.4897 - val_head_pi_loss: 1.9459
Epoch 14/20
30/30 [==============================] - 0s 797us/sample - loss: 2.4401 - head_value_loss: 0.4942 - head_pi_loss: 1.9459 - val_loss: 2.4050 - val_head_v

[0.1405225694179535, 0.14405618607997894, 0.14308829605579376, 0.13856175541877747, 0.15043488144874573, 0.14323465526103973, 0.1401015669107437]
[[3, 0], [2, 1], [5, 2], [0, 3], [2, 4], [3, 5], [5, 6]]
[5, 2]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  1.  0. -1. -1.  0.  0.]
 [ 1.  1.  0. -1.  1. -1.  0.]
 [ 1.  1. -1.  1. -1. -1.  0.]]
[0.14101383090019226, 0.1443358063697815, 0.14268894493579865, 0.13896001875400543, 0.1503884196281433, 0.14218953251838684, 0.1404235064983368]
[[3, 0], [2, 1], [4, 2], [0, 3], [2, 4], [3, 5], [5, 6]]
[3, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  1.  0. -1. -1.  1.  0.]
 [ 1.  1.  0. -1.  1. -1.  0.]
 [ 1.  1. -1.  1. -1. -1.  0.]]
[0.14039403200149536, 0.1440139263868332, 0.143052339553833, 0.13865330815315247, 0.15032528340816498, 0.14328926801681519, 0.1402718722820282]
[[3, 0], [2, 1], [4, 2], [0, 3], [2, 4], [2, 5], [5, 6]]
[5, 6]

...

[0.13916011154651642, 0.14595244824886322, 0.14329968392848969, 0.1352589726448059, 0.15461435914039612, 0.1425236314535141, 0.13919079303741455]
[[4, 0], [3, 1], [3, 2], [2, 3], [1, 4], [1, 5], [3, 6]]
[2, 3]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  1.  0.]
 [ 0.  0.  0. -1. -1. -1.  0.]
 [ 0. -1. -1.  1. -1.  1.  1.]
 [-1.  1.  1. -1.  1.  1.  1.]]
[0.1381670981645584, 0.1451437920331955, 0.14278537034988403, 0.13601601123809814, 0.15471991896629333, 0.14352020621299744, 0.13964752852916718]
[[4, 0], [3, 1], [3, 2], [1, 3], [1, 4], [1, 5], [3, 6]]
[4, 0]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  1.  0.]
 [ 0.  0.  0. -1. -1. -1.  0.]
 [-1. -1. -1.  1. -1.  1.  1.]
 [-1.  1.  1. -1.  1.  1.  1.]]
[0.13937929272651672, 0.14605015516281128, 0.14340269565582275, 0.13537222146987915, 0.15421393513679504, 0.14240144193172455, 0.13918030261993408]
[[3, 0], [3, 1], [3, 2], [1, 3], [1, 4], [1, 5], [3, 6]]
[

[0.13886143267154694, 0.1449919492006302, 0.1430719643831253, 0.1365966945886612, 0.15315760672092438, 0.14354804158210754, 0.13977238535881042]
[[5, 0], [4, 1], [3, 2], [3, 3], [5, 4], [4, 5], [4, 6]]
[4, 6]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0. -1.]
 [ 0. -1.  1.  1.  0.  1. -1.]]
[0.13954712450504303, 0.14587239921092987, 0.14361491799354553, 0.1360722780227661, 0.1530880630016327, 0.14196182787418365, 0.13984335958957672]
[[5, 0], [4, 1], [3, 2], [3, 3], [5, 4], [4, 5], [3, 6]]
[5, 4]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0. -1.]
 [ 0. -1.  1.  1.  1.  1. -1.]]
(9, 3, 6, 7)
(9, 1)
(9, 7)
Train on 7 samples, validate on 2 samples
Epoch 1/20
7/7 [==============================] - 0s 9ms/sample - loss: 2.6876 - head_value_loss: 0.7423 - head_pi_loss: 1.9453 - val_loss

[0.1386486440896988, 0.14526072144508362, 0.14299626648426056, 0.1359899342060089, 0.15401960909366608, 0.1435568779706955, 0.1395280361175537]
[[5, 0], [4, 1], [3, 2], [3, 3], [4, 4], [4, 5], [3, 6]]
[3, 3]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.  1.]
 [ 0. -1. -1.  1.  1.  1. -1.]]
[0.13950134813785553, 0.14624032378196716, 0.14326797425746918, 0.13570357859134674, 0.15396946668624878, 0.14172790944576263, 0.13958938419818878]
[[5, 0], [4, 1], [3, 2], [2, 3], [4, 4], [4, 5], [3, 6]]
[5, 0]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.  1.]
 [ 1. -1. -1.  1.  1.  1. -1.]]
[0.1385708749294281, 0.14525067806243896, 0.14299866557121277, 0.13592815399169922, 0.15403425693511963, 0.14359058439731598, 0.13962677121162415]
[[4, 0], [4, 1], [3, 2], [2, 3], [4, 4], [4, 5], [3, 6]]
[4

[0.1382949948310852, 0.1454467624425888, 0.14302299916744232, 0.13568606972694397, 0.15439188480377197, 0.14363697171211243, 0.13952034711837769]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [4, 5], [5, 6]]
[4, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
[0.13953351974487305, 0.14608463644981384, 0.14305144548416138, 0.13526706397533417, 0.15449655055999756, 0.14218276739120483, 0.1393839716911316]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [3, 5], [5, 6]]
[5, 3]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  1.  0.  1.  0.]]
[0.13816247880458832, 0.14546085894107819, 0.14290395379066467, 0.13591253757476807, 0.15436764061450958, 0.1434713900089264, 0.13972118496894836]
[[5, 0], [5, 1], [5, 2], [4, 3], [5, 4], [3, 5], [5, 6]]
[

[0.1392035037279129, 0.1460241973400116, 0.14300434291362762, 0.13570593297481537, 0.15493443608283997, 0.1418565958738327, 0.13927103579044342]
[[4, 0], [3, 1], [2, 2], [3, 4], [3, 6]]
[3, 6]
[[ 0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0. -1. -1.  0. -1.  1.]
 [ 0. -1.  1.  1. -1. -1. -1.]
 [ 1.  1. -1.  1. -1.  1.  1.]]
[0.13806378841400146, 0.1452706903219223, 0.14331217110157013, 0.1353960782289505, 0.154572531580925, 0.1439460664987564, 0.13943864405155182]
[[4, 0], [3, 1], [2, 2], [3, 4], [2, 6]]
[2, 2]
[[ 0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0. -1. -1.  0. -1.  0.]
 [ 0.  0. -1. -1.  0. -1.  1.]
 [ 0. -1.  1.  1. -1. -1. -1.]
 [ 1.  1. -1.  1. -1.  1.  1.]]
[0.1389549970626831, 0.14602957665920258, 0.1429130882024765, 0.1358000487089157, 0.1552170217037201, 0.14183664321899414, 0.13924863934516907]
[[4, 0], [3, 1], [1, 2], [3, 4], [2, 6]]
[2, 6]
[[ 0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0.  1.  

In [493]:
game = connect_four_game(6,7)
v = mcts.search(np.concatenate((game.channel_0,game.channel_1,game.channel_2),axis=0),game,network,-1)
p = mcts.p[str(mcts.explored[-1])]
states = mcts.full_state
for state in states:
    network.accuracy(state,v,p)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


acc is : 1.0
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]


In [522]:
mcts.win_rate(network)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[5, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]]
[0.1445310264825821, 0.14365050196647644, 0.14498092234134674, 0.13923969864845276, 0.14207836985588074, 0.14413833618164062, 0.14138120412826538]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [4, 5], [5, 6]]
[5, 6]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  1.]]
[0.14439845085144043, 0.14214138686656952, 0.14295990765094757, 0.1414768248796463, 0.1435345709323883, 0.14270927011966705, 0.1427796334028244]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [4, 5], [4, 6]]
[5, 2]
[

[0.14471866190433502, 0.1418488770723343, 0.14255386590957642, 0.1418328434228897, 0.14360089600086212, 0.14262239634990692, 0.14282245934009552]
[[5, 0], [4, 1], [4, 2], [3, 3], [5, 4], [3, 5], [5, 6]]
[5, 6]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.]
 [ 0.  1. -1.  1.  0.  1. -1.]]
[0.14445842802524567, 0.14405347406864166, 0.14545206725597382, 0.13955572247505188, 0.14144861698150635, 0.14371608197689056, 0.14131557941436768]
[[5, 0], [4, 1], [4, 2], [3, 3], [5, 4], [3, 5], [4, 6]]
[5, 0]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.]
 [ 1.  1. -1.  1.  0.  1. -1.]]
[0.1447535753250122, 0.14191822707653046, 0.1424076408147812, 0.14164778590202332, 0.1436387002468109, 0.14273157715797424, 0.14290250837802887]
[[4, 0], [4, 1], [4, 2], [3, 3], [5, 4], [3, 5], [4, 6]]
[4

[0.14462034404277802, 0.14412161707878113, 0.1450791209936142, 0.13906946778297424, 0.1421646624803543, 0.14417290687561035, 0.14077189564704895]
[[5, 0], [2, 1], [5, 2], [5, 3], [2, 4], [1, 5], [5, 6]]
[5, 2]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0. -1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  0.  1. -1.  0.]
 [ 0.  1.  1.  0. -1. -1.  0.]]
[0.1440243422985077, 0.1426294595003128, 0.14327724277973175, 0.1408005803823471, 0.14360055327415466, 0.14340338110923767, 0.1422644406557083]
[[5, 0], [2, 1], [4, 2], [5, 3], [2, 4], [1, 5], [5, 6]]
[5, 3]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0. -1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  0.  1. -1.  0.]
 [ 0.  1.  1. -1. -1. -1.  0.]]
[0.1450137048959732, 0.14425794780254364, 0.1450340449810028, 0.13930298388004303, 0.14184480905532837, 0.143646240234375, 0.14090028405189514]
[[5, 0], [2, 1], [4, 2], [4, 3], [2, 4], [1, 5], [5, 6]]
[2, 4]


[0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548, 0.1428571492433548]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[5, 4]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]
[0.14442503452301025, 0.14181968569755554, 0.14336153864860535, 0.14119446277618408, 0.14381465315818787, 0.1431727409362793, 0.14221185445785522]
[[5, 0], [5, 1], [5, 2], [5, 3], [4, 4], [5, 5], [5, 6]]
[5, 2]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  1.  0.  0.]]
[0.14446204900741577, 0.14364998042583466, 0.14512194693088531, 0.13935743272304535, 0.1419389843940735, 0.14405544102191925, 0.14141413569450378]
[[5, 0], [5, 1], [4, 2], [5, 3], [4, 4], [5, 5], [5, 6]]
[4, 4]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 

...

[0.14454934000968933, 0.14323025941848755, 0.14510475099086761, 0.13952277600765228, 0.14213134348392487, 0.1440153270959854, 0.14144615828990936]
[[4, 0], [5, 1], [4, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[4, 2]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [-1.  0.  1.  0.  0.  0.  0.]]
[0.1442817747592926, 0.14188650250434875, 0.14285732805728912, 0.1417727917432785, 0.1437751203775406, 0.14286358654499054, 0.1425628364086151]
[[4, 0], [5, 1], [3, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[4, 0]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  1.  0.  0.  0.  0.]
 [-1.  0.  1.  0.  0.  0.  0.]]
[0.14459072053432465, 0.1429673433303833, 0.14489483833312988, 0.13981281220912933, 0.14255385100841522, 0.143841952085495, 0.1413385570049286]
[[3, 0], [5, 1], [3, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[5, 5]

[0.1445169448852539, 0.1420646607875824, 0.14304609596729279, 0.14090070128440857, 0.14371030032634735, 0.14311480522155762, 0.14264647662639618]
[[4, 0], [5, 1], [4, 2], [4, 3], [2, 4], [5, 6]]
[5, 1]
[[ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  0.  0.  1.  1.  0.]
 [-1. -1.  1.  1. -1. -1.  0.]]
[0.14556707441806793, 0.14384832978248596, 0.14465782046318054, 0.1393214762210846, 0.1420225203037262, 0.1435713768005371, 0.14101140201091766]
[[4, 0], [4, 1], [4, 2], [4, 3], [2, 4], [5, 6]]
[4, 2]
[[ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  1.  0.  1.  1.  0.]
 [-1. -1.  1.  1. -1. -1.  0.]]
[0.14462149143218994, 0.14204159379005432, 0.1429814100265503, 0.1413811445236206, 0.14341747760772705, 0.1428106278181076, 0.1427462100982666]
[[4, 0], [4, 1], [3, 2], [4, 3], [2, 4], [5, 6]]
[3, 2]
[[ 0.  0.  0.  0.  0.  

[0.14489182829856873, 0.14197847247123718, 0.14233842492103577, 0.14141298830509186, 0.1438763588666916, 0.14267444610595703, 0.14282749593257904]
[[3, 0], [4, 1], [5, 2], [4, 3], [3, 4], [4, 5], [1, 6]]
[4, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0. -1.]
 [-1.  0.  0.  0.  1. -1.  1.]
 [-1.  1.  0. -1. -1.  1.  1.]]
[0.14501559734344482, 0.1434817761182785, 0.14466527104377747, 0.13968980312347412, 0.14168600738048553, 0.14403825998306274, 0.14142322540283203]
[[3, 0], [4, 1], [5, 2], [4, 3], [3, 4], [3, 5], [1, 6]]
[3, 0]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0. -1.]
 [-1.  0.  0.  0.  1. -1.  1.]
 [-1.  1.  0. -1. -1.  1.  1.]]
[0.14507101476192474, 0.14199233055114746, 0.14214304089546204, 0.14161476492881775, 0.143877312541008, 0.14256462454795837, 0.14273680746555328]
[[2, 0], [4, 1], [5, 2], [4, 3], [3, 4], [3, 5], [1, 6]]
[

[0.14483599364757538, 0.14215242862701416, 0.1430174857378006, 0.14130844175815582, 0.1432100236415863, 0.14253298938274384, 0.1429426372051239]
[[5, 0], [4, 1], [4, 2], [3, 3], [3, 4], [4, 5], [3, 6]]
[5, 0]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0. -1.]
 [-1. -1.  1. -1. -1.  1.  1.]]
[0.14443349838256836, 0.1438877433538437, 0.14547143876552582, 0.13939045369625092, 0.14184686541557312, 0.1436818540096283, 0.1412881761789322]
[[4, 0], [4, 1], [4, 2], [3, 3], [3, 4], [4, 5], [3, 6]]
[3, 4]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  1.  0. -1.]
 [-1. -1.  1. -1. -1.  1.  1.]]
[0.14483173191547394, 0.14215078949928284, 0.14306072890758514, 0.14113877713680267, 0.14339153468608856, 0.14242614805698395, 0.14300034940242767]
[[4, 0], [4, 1], [4, 2], [3, 3], [2, 4], [4, 5], [3, 6]]
[4,

0.8

In [12]:
mcts.explored

[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0., -1.,  0.,  0.],
        [-1.,  1., -1., -1.,  1.,  0.,  0.]]),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0., -1.,  0.,  0.],
        [-1.,  1., -1., -1.,  1.,  0.,  0.]]),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0., -1.,  0.,  0.],
        [-1.,  1., -1., -1.,  1.,  0.,  0.]]),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
    

In [14]:
mcts.p

{'[[0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0.]]': [0.14003397524356842,
  0.14548847079277039,
  0.14313821494579315,
  0.1373085230588913,
  0.1508772224187851,
  0.1423332691192627,
  0.14082030951976776],
 '[[ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0. -1.  0.  0.  0.  0.]]': [0.14114224910736084,
  0.1445312350988388,
  0.1429695188999176,
  0.13807837665081024,
  0.15081942081451416,
  0.1425842046737671,
  0.13987506926059723],
 '[[ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0.]\n [ 0.  1. -1.  0.  0.  0.  0.]]': [0.1398918777704239,
  0.1442106068134308,
  0.14277955889701843,
  0.13718059659004211,
  0.15252245962619781,
  0.14369617402553558,
  0

In [20]:
game = connect_four_game(6,7)
mcts.play_human(game,network,1)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


[0.14042456448078156, 0.14514975249767303, 0.14319562911987305, 0.1379045695066452, 0.15000192821025848, 0.1423892080783844, 0.14093436300754547]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
[5, 5]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]]
Enter drop column for Player 1: 0
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0. -1.  0.]]
[0.13893599808216095, 0.14514978229999542, 0.14308415353298187, 0.13651826977729797, 0.15312977135181427, 0.14342080056667328, 0.13976125419139862]
[[4, 0], [5, 1], [5, 2], [5, 3], [5, 4], [4, 5], [5, 6]]
[5, 3]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0. -

([array([[[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.]]]),
  array([[[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0.